# RAG

Retrieval-Augmented Generation (RAG) is a model that combines the strengths of retrieval and generation models. It uses a retriever to find relevant passages and a generator to generate the answer. The retriever is a dense retriever that uses a pre-trained language model to encode the passages and the query. The generator is a pre-trained language model that generates the answer. The retriever and the generator are trained jointly.

## Process

1. The retriever encodes the passages and the query.
2. The retriever retrieves the top-k passages.
3. The generator generates the answer using the top-k passages and reture results based on conditions.


### Load data


Add support for loading data from the `datasets` library.

### Data Preprocessing

The data is preprocessed to remove special characters and convert the text to lowercase.

### Embedding

The retriever uses a pre-trained language model to encode the passages and the query. The generator uses a pre-trained language model to generate the answer.

### Reedtrieval

The retriever retrieves the top-k passages using the encoded passages and the query.

### Augmented Generation 

Bae on the retrieved passages, the generator generates the answer.


## Sample

In [ ]:
import { TextLoader } from 'langchain/document_loaders/fs/text'


// load the documents
const loader = new TextLoader('data/qiu.txt')
const docs = await loader.load()

console.log(docs)

In [ ]:
// split the documents into sentences

import { RecursiveCharacterTextSplitter } from 'langchain/text_splitter'

const splitter = new RecursiveCharacterTextSplitter({
    chunkSize: 500,
    chunkOverlap: 100 // overlap of 100 characters will help to understand the context of the sentence
})

const splitDocs = await splitter.splitDocuments(docs)
console.log(splitDocs[4])

In [ ]:
// import env config

import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [4]:
// vertor store and retrieval

import { OpenAIEmbeddings } from '@langchain/openai'

const embeddings = new OpenAIEmbeddings()

import { MemoryVectorStore } from 'langchain/vectorstores/memory'

const vectorStore = new MemoryVectorStore(embeddings)

await vectorStore.addDocuments(splitDocs)

In [5]:
// Retrieve the vectors
const retriever = vectorStore.asRetriever(2)

const query = "原文中，谁提出了宏原子的假设？并详细介绍给我宏原子假设的理论?"

const results = await retriever.invoke(query)

console.log(results)

[
  Document {
    pageContent: "“那么，像刚才说的，是否存在宏质子和宏中子呢？”\n" +
      "　　“应该存在，不过由于它们不能被激发，我们很难发现它们。”\n" +
      "　　“丁教授，你的梦实现了。”林云说，除了丁仪和我，别的人还不太明白她这话的意思。\n" +
      "　　"... 370 more characters,
    metadata: {
      source: "data/qiu.txt",
      loc: { lines: { from: 1465, to: 1474 } }
    }
  },
  Document {
    pageContent: "“宏原子核？”\n" +
      "　　“是的，通过观测一个宏电子在空间中的运动，借助这个数学模型，我们就能精确定位这个宏电子对应的原子核的准确位置。”\n" +
      "　　“可我们怎么样才能探测到那个原子核呢？”\n" +
      "　　“同宏电子一样，"... 356 more characters,
    metadata: {
      source: "data/qiu.txt",
      loc: { lines: { from: 2205, to: 2220 } }
    }
  }
]


In [6]:
import { RunnableSequence } from '@langchain/core/runnables'

const converDocsToString = (docs) => {
    return docs.map(doc => doc.pageContent).join('\n')
}
const contextRetriverChain = RunnableSequence.from([
    (input) => input.question,
    retriever,
    converDocsToString
])

const result = await contextRetriverChain.invoke({ question: query })

console.log(result)

“那么，像刚才说的，是否存在宏质子和宏中子呢？”
　　“应该存在，不过由于它们不能被激发，我们很难发现它们。”
　　“丁教授，你的梦实现了。”林云说，除了丁仪和我，别的人还不太明白她这话的意思。
　　“是啊是啊，真有西瓜这么大的基本粒子摆上物理学家的桌面了，下一步我们肯定要研究它们的内部结构，那也是由弯曲的空间构成的结构，虽然也很难，但我相信比研究微观粒子的结构不知要容易多少倍。”
　　“那也存在宏原子了？三种宏粒子应该是能够组成原子的啊！”
　　“是的，应该有宏原子。”
　　“我们所捕获到的那个空泡，哦，那个宏电子，是自由电子呢，还是一个宏原子中的电子？如果是后者，那这个宏原子的原子核在哪里呢？”
　　“呵呵，您问住我了。不过，原子中的空间很大，如果一个原子有一个剧场大厅那么大，原子核只是大厅中央的一个核桃大小，所以，如果这个宏电子真的属于一个宏原子，那它的原子核距离我们是相当远的。”
　　“天啊，还有一个大问题：如果存在宏原子，那一定有宏物质，也有宏世界了？”
　　“我们已经在进行宏伟的哲学思考了。”丁仪向提问者微笑着说。
“宏原子核？”
　　“是的，通过观测一个宏电子在空间中的运动，借助这个数学模型，我们就能精确定位这个宏电子对应的原子核的准确位置。”
　　“可我们怎么样才能探测到那个原子核呢？”
　　“同宏电子一样，这事情同样惊人地简单：我们能用肉眼看到它。”
　　“哇……它看上去是什么样儿？你好像说过，原子核的外形与宏电子的空泡形状完全不同。”
　　“弦。”
　　“弦？”
　　“对，一根弦，它看上去是一根弦。”
　　“多长多粗的弦呢？”
　　“它与宏电子基本处于一个尺度级别，长度大约在一到两米之间，依原子的种类不同而异，至于粗细，弦是无限细的，它上面的每一点都是没有大小的奇点。”
　　“我们怎么可能用肉眼看到一根无限细的弦？”
　　“因为光线在它的附近同样会发生弯曲。”
　　“那它看上去是什么样子呢？”
　　丁仪半闭着双眼，仿佛一个刚刚睡醒的人在回忆着刚才的梦，“它看上去，就像一条透明的水晶蛇，像一根无法自缢的绳索。”
　　“后一个比喻好奇怪。”
　　“因为这根弦已经是组成宏物质的最小单位，它是不可能被剪断的。”


In [7]:
// Generate the template

import { ChatPromptTemplate } from '@langchain/core/prompts'

const TEMPLATE = `
你是一个熟读刘慈欣的《球状闪电》的终极原著党，精通根据作品原文详细解释和回答问题，你在回答时会引用作品原文。
并且回答时仅根据原文，尽可能回答用户问题，如果原文中没有相关内容，你可以回答“原文中没有相关内容”，

以下是原文中跟用户回答相关的内容：
{context}

现在，你需要基于原文，回答以下问题：
{question}`;

const prompt = ChatPromptTemplate.fromTemplate(TEMPLATE)

In [8]:
import { ChatOpenAI } from '@langchain/openai'
import { StringOutputParser } from "@langchain/core/output_parsers"

const model = new ChatOpenAI()

const ragChain = RunnableSequence.from([
    {
        context: contextRetriverChain,
        question: input => input.question
    },
    prompt,
    model,
    new StringOutputParser()
])

const answer = await ragChain.invoke({ question: query })

console.log(answer)


原文中，丁教授提出了宏原子的假设。他认为宏原子存在，并通过观测宏电子在空间中的运动，可以精确定位宏电子对应的原子核的位置。宏原子的外形是一根无限细的弦，长度大约在一到两米之间，依据原子的种类而有所不同。弦上的每一点都是没有大小的奇点，它是组成宏物质的最小单位，不可能被剪断。光线在宏原子附近同样会发生弯曲，因此我们可以通过肉眼观察到它。宏原子的外观类似透明的水晶蛇或无法自缢的绳索。


In [9]:
const answer = await ragChain.invoke({
    question: "详细描述原文中有什么跟直升机相关的场景"
});

console.log(answer);

在原文中，直升机进行了一次试验飞行。根据计划，直升机在达到预定高度后，会点燃电弧并顺风飞行一段距离，然后熄灭电弧并返回重复上述过程。在顺风飞行中，两架直升机相互靠近，螺旋桨边缘几乎碰到一起，形成一道明亮的电弧，并发出清脆的噼啪声。随着距离的增大，电弧的波动也越来越大，最后两架直升机到达极限位置时，电弧仿佛是一条在风中狂舞的轻纱。整个场景在暗蓝色的晨空背景中显得不真实，给人一种电影胶片被划痕的感觉。
